# Setup


In [ ]:
# %%capture
# %pip install langchain==0.3.27
# %pip install jupyterlab==4.4.9

In [ ]:
import os
import time
import uuid
import operator
from typing import TypedDict, Annotated
from dotenv import load_dotenv
from IPython.display import Image

__import__("pysqlite3")
import sys

sys.modules["sqlite3"] = sys.modules.pop("pysqlite3")

import pandas as pd

from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.tools import tool, create_retriever_tool
from langchain_core.messages import AnyMessage, AIMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.document_loaders import PyPDFLoader
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.prebuilt import create_react_agent
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

In [ ]:
load_dotenv()

GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]

In [ ]:
# MODELS
model_gemini_2_0_flash = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=GOOGLE_API_KEY,
    temperature=0,
    max_tokens=100,
    max_retries=1,
)

# model_embedding_001 = GoogleGenerativeAIEmbeddings(
#     model="models/embedding-001", google_api_key=GOOGLE_API_KEY
# )

model_embedding_001 = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2", model_kwargs={"device": "cpu"}
)

# ReAct Agent Example


## Create agent


In [ ]:
@tool
def find_sum(a: int, b: int) -> int:
    """
    Add two integers and return their sum.
    """
    return a + b


@tool
def find_product(a: int, b: int) -> int:
    """
    Multiply two integers and return their product.
    """
    return a * b

In [ ]:
agent_tools = [find_sum, find_product]

system_prompt = SystemMessage(
    """
    You are a mathematician who solves math problems using tools only. 
    Do not solve any problem yourself — always use the available tools to find the solution.
    """
)

agent_graph = create_react_agent(
    model=model_gemini_2_0_flash, prompt=system_prompt, tools=agent_tools
)

## Invoke agent


In [ ]:
# # Example 1
# inputs = {"messages": [("user", "what is the sum of 2 and 3 ?")]}
# result = agent_graph.invoke(inputs)

# print(f"Agent returned : {result['messages'][-1].content} \n")
# print("Step by Step execution : ")
# for message in result["messages"]:
#     print(message.pretty_repr())

In [ ]:
# # Example 2
# inputs = {"messages": [("user", "What is 3 multipled by 2 and 5 + 1 ?")]}
# result = agent_graph.invoke(inputs)

# print(f"Agent returned : {result['messages'][-1].content} \n")
# print("Step by Step execution : ")
# for message in result["messages"]:
#     print(message.pretty_repr())

In [ ]:
# # Example 3
# inputs = {"messages": [("user", "what is the sum of 2.1 and 3.7 ?")]}
# result = agent_graph.invoke(inputs)

# print(f"Agent returned : {result['messages'][-1].content} \n")
# print("Step by Step execution : ")
# for message in result["messages"]:
#     print(message.pretty_repr())

## Debug agent


In [ ]:
agent_graph_debug = create_react_agent(
    model=model_gemini_2_0_flash, prompt=system_prompt, tools=agent_tools, debug=True
)

In [ ]:
# inputs = {"messages": [("user", "what is the sum of 2 and 3 ?")]}
# result = agent_graph_debug.invoke(inputs)

# Product Q&A Chatbot


## Data


In [ ]:
product_price_df = pd.read_csv("data/smartphone_prices.csv")
product_price_df

In [ ]:
loader = PyPDFLoader("./data/smartphone_descriptions.pdf")
docs = loader.load()
docs

## Tools


In [ ]:
@tool
def get_smartphone_price(smartphone_name: str) -> int:
    """
    Returns the price of a smartphone by name (case-insensitive substring match).
    If no match is found, returns -1.
    """
    pattern = f"^{smartphone_name.strip()}"
    matches = product_price_df[
        product_price_df["Name"].str.contains(pattern, case=False, na=False)
    ]
    if matches.empty:
        return -1
    return int(matches["Price"].iloc[0])


print(get_smartphone_price.invoke("zenith"))
print(get_smartphone_price.invoke("asdf"))

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256)
splits = text_splitter.split_documents(docs)

feature_store = Chroma.from_documents(documents=splits, embedding=model_embedding_001)

get_product_features = create_retriever_tool(
    feature_store.as_retriever(search_kwargs={"k": 1}),
    name="Get_Product_Features",
    description="""
    This store contains details about smartphones. It lists the available smartphones
    and their features including camera, memory, storage, design and advantages
    """,
)

feature_store.as_retriever().invoke("Tell me about the Zenith One")

## Chatbot

In [ ]:
system_prompt = SystemMessage(
    """
    You are professional chatbot that answers questions about smartphones sold by your company.
    To answer questions about smartphones, you will ONLY use the available tools and NOT your own memory.
    You will handle small talk and greetings by producing professional responses.
    """
)
checkpointer = MemorySaver()  # conversation memory

product_QnA_agent = create_react_agent(
    model=model_gemini_2_0_flash,
    tools=[get_smartphone_price, get_product_features],
    prompt=system_prompt,
    debug=False,
    checkpointer=checkpointer,
)

In [ ]:
# To maintain memory, each request should be in the context of a thread.
# Each user conversation will use a separate thread ID
config = {"configurable": {"thread_id": uuid.uuid4()}}

inputs = {
    "messages": [HumanMessage("What are the features and pricing for Zenith One?")]
}

# Use streaming to print responses as the agent  does the work.
# This is an alternate way to stream agent responses without waiting for the agent to finish
for stream in product_QnA_agent.stream(inputs, config, stream_mode="values"):
    message = stream["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

## Execute

In [ ]:
# This simulates the conversation between the user and the Agentic chatbot
user_inputs = [
    "Hello",
    "I am looking to buy a smartphone",
    "Give me a list of available smartphone names",
    "Tell me about the features of Zenith One",
    "How much does it cost?",
    "Give me similar information about TitanMax",
    "What info do you have on Nimbus ?",
    "Thanks for the help",
]

# Create a new thread
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

for input in user_inputs:
    time.sleep(1)
    print(f"----------------------------------------\nUSER : {input}")
    user_message = {"messages": [HumanMessage(input)]}
    ai_response = product_QnA_agent.invoke(user_message, config=config)
    print(f"AGENT : {ai_response['messages'][-1].content}")

In [ ]:
# conversation memory by user
def execute_prompt(user, config, prompt):
    inputs = {"messages": [("user", prompt)]}
    ai_response = product_QnA_agent.invoke(inputs, config=config)
    print(f"\n{user}: {ai_response['messages'][-1].content}")


# Create different session threads for 2 users
config_1 = {"configurable": {"thread_id": str(uuid.uuid4())}}
config_2 = {"configurable": {"thread_id": str(uuid.uuid4())}}

# Test both threads
execute_prompt("USER 1", config_1, "Tell me about the features of  Zenith")
execute_prompt("USER 2", config_2, "Tell me about the features of Nimbus Mini")
execute_prompt("USER 1", config_1, "What is its price ?")
execute_prompt("USER 2", config_2, "What is its price ?")

# Orders Chatbot

## Data

In [ ]:
product_orders_df = pd.read_csv("data/smartphone_orders.csv")
product_orders_df

## Tools

In [ ]:
@tool
def get_order_details(order_id: str) -> str:
    """
    This function returns details about a smartphone order, given an order ID
    It performs an exact match between the input order id and available order ids
    If a match is found, it returns products (smartphones) ordered, quantity ordered and delivery date.
    If there is NO match found, it returns -1
    """
    matches = product_orders_df[product_orders_df["Order ID"] == order_id]

    if matches.empty:
        return -1
    else:
        return matches.iloc[0].to_dict()


print(get_order_details.invoke("ORD-5821"))
print(get_order_details.invoke("ORD-8510"))

In [ ]:
@tool
def update_quantity(order_id: str, new_quantity: int) -> bool:
    """
    This function updates the quantity of products (smartphones) ordered for a given order Id.
    It there are no matching orders, it returns -1.
    """
    matches = product_orders_df[product_orders_df["Order ID"] == order_id]

    if matches.empty:
        return -1
    else:
        product_orders_df.loc[
            product_orders_df["Order ID"] == order_id, "Quantity Ordered"
        ] = new_quantity
        return True

In [ ]:
# # to test comment out @tool decorator
# print(get_order_details("ORD-5821"))
# print(update_quantity("ORD-5821", 20))
# print(get_order_details("ORD-9022"))
# print(update_quantity("ORD-9022", 50))
# product_orders_df

## Chatbot

In [ ]:
# An Agent State class that keeps the state of the agent while it answers a query
class OrdersAgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]


# An Agent class
class OrdersAgent:

    def __init__(self, model, tools, system_prompt, debug):

        self.system_prompt = system_prompt
        self.debug = debug

        # Setup the graph for the agent manually
        agent_graph = StateGraph(OrdersAgentState)
        agent_graph.add_node("orders_llm", self.call_llm)
        agent_graph.add_node("orders_tools", self.call_tools)
        agent_graph.add_conditional_edges(
            "orders_llm", self.is_tool_call, {True: "orders_tools", False: END}
        )
        agent_graph.add_edge("orders_tools", "orders_llm")
        agent_graph.set_entry_point("orders_llm")
        self.memory = MemorySaver()  # chat memory
        self.agent_graph = agent_graph.compile(
            checkpointer=self.memory
        )  # compile the graph

        # Setup tools
        self.tools = {tool.name: tool for tool in tools}
        if self.debug:
            print("\nTools loaded :", self.tools)
        self.model = model.bind_tools(tools)  # attach tools to model

    # Call the LLM with the messages to get next action/result
    def call_llm(self, state: OrdersAgentState):

        messages = state["messages"]
        if self.system_prompt:
            messages = [SystemMessage(content=self.system_prompt)] + messages

        result = self.model.invoke(
            messages
        )  # invoke the model with the message history
        if self.debug:
            print(f"\nLLM Returned : {result}")

        return {"messages": [result]}  # Return the LLM output

    # Check if the next action is a tool call.
    def is_tool_call(self, state: OrdersAgentState):
        last_message = state["messages"][-1]
        if len(last_message.tool_calls) > 0:
            return True
        else:
            return False

    # Execute the tool requested with the given parameters
    def call_tools(self, state: OrdersAgentState):
        tool_calls = state["messages"][-1].tool_calls
        results = []

        for tool in tool_calls:
            # Handle tool missing error
            if not tool["name"] in self.tools:
                print(f"Unknown tool name {tool}")
                result = "Invalid tool found. Please retry"
            else:
                result = self.tools[tool["name"]].invoke(tool["args"])

            # append results to the list of tool results
            results.append(
                ToolMessage(
                    tool_call_id=tool["id"], name=tool["name"], content=str(result)
                )
            )

            if self.debug:
                print(f"\nTools returned {results}")
            # return tool results
            return {"messages": results}

In [ ]:
system_prompt = """
    You are professional chatbot that manages orders for smartphones sold by our company.
    The tools allow for retrieving order details as well as update order quantity.
    Do NOT reveal information about other orders than the one requested.
    You will handle small talk and greetings by producing professional responses.
    """

orders_agent = OrdersAgent(
    model_gemini_2_0_flash,
    [get_order_details, update_quantity],
    system_prompt,
    debug=False,
)

# Visualize the Agent
Image(orders_agent.agent_graph.get_graph().draw_mermaid_png())

## Execute

In [ ]:
user_inputs = [
    "How are you doing?",
    "Please show me the details of the order ORD-5821",
    "Can you add one more of that laptop to the order? ",
    "Can you show me the details again ? ",
    "What about order ORD-9999 ?",
    "Bye",
]

config = {"configurable": {"thread_id": str(uuid.uuid4())}}  # create a new thread

for input in user_inputs:
    print(f"----------------------------------------\nUSER : {input}")
    user_message = {"messages": [HumanMessage(input)]}  # format the user message
    ai_response = orders_agent.agent_graph.invoke(
        user_message, config=config
    )  # get response from the agent
    print(f"\nAGENT : {ai_response['messages'][-1].content}")

# Reflection-based Summary Agent

In [ ]:
STOP_MESSAGE = "DECISION: STOP"
CONTINUE_MESSAGE = "DECISION: CONTINUE"

summarizer_prompt = """
You are a document summarizer. Summarize provided text in under 100 words.
If human or ai gives review, feedback or critique, revise your previous summary accordingly and provide improved summary.
"""

reviewer_prompt = f"""
You are a reviewer grading summaries.

1. Read through the messages and find the original text and the last summary.
2. Give brief, clear feedback about accuracy, coverage, and brevity.
3. You MUST always output at least one sentence of feedback.
4. At the end of your feedback, output exactly one of the following lines:
{STOP_MESSAGE}
{CONTINUE_MESSAGE}
"""

In [ ]:
class SummaryAgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]


class SummaryAgent:

    def __init__(self, model, summarizer_prompt, reviewer_prompt, debug):
        self.model = model
        self.summarizer_prompt = summarizer_prompt
        self.reviewer_prompt = reviewer_prompt
        self.debug = debug

        # Setup the graph for the agent manually
        agent_graph = StateGraph(SummaryAgentState)
        agent_graph.add_node("summarizer", self.generate_summary)
        agent_graph.add_node("reviewer", self.review_summary)
        agent_graph.add_conditional_edges(
            "summarizer", self.should_continue, {True: "reviewer", False: END}
        )
        agent_graph.add_edge("reviewer", "summarizer")
        agent_graph.set_entry_point("summarizer")
        self.memory = MemorySaver()  # chat memory
        self.agent_graph = agent_graph.compile(
            checkpointer=self.memory
        )  # compile the graph

    def generate_summary(self, state: SummaryAgentState):
        messages = state["messages"]
        messages = [SystemMessage(content=self.summarizer_prompt)] + messages

        # Invoke summarizer with the message history
        result = self.model.invoke(messages)

        if self.debug:
            print("--- Prompt for summarizer ---")
            for m in messages:
                print(f"{m.type.upper()}: {m.content}")
            print("-----------------------------")
            print(
                f"==============\nSummarizer output:\n{result.content}\n==============\n"
            )
        tagged_result = AIMessage(content=f"[SUMMARY]\n{result.content}")
        return {"messages": [tagged_result]}

    def review_summary(self, state: SummaryAgentState):
        messages = state["messages"]
        messages = [SystemMessage(content=self.reviewer_prompt)] + messages

        # Invoke reviewer with the message history
        result = self.model.invoke(messages)

        if self.debug:
            print(f"*************\nReviewer output:\n{result.content}\n*************\n")
        tagged_result = AIMessage(content=f"[REVIEW]\n{result.content}")
        return {"messages": [tagged_result]}

    def should_continue(self, state: SummaryAgentState):
        total_reviews = len(state["messages"]) / 2
        last_review = next(
            (
                m
                for m in reversed(state["messages"])
                if m.content.startswith("[REVIEW]")
            ),
            "",
        )
        if self.debug:
            print(f"Iteration number: {total_reviews}\n")
            print(f"Last review: {last_review}")

        # Return if 2 iterations are completed or summary is satisfactory.
        # Each iteration has 2 messages
        if len(state["messages"]) > 4 or STOP_MESSAGE in last_review:
            return False
        else:
            return True

In [ ]:
summary_chatbot = SummaryAgent(
    model_gemini_2_0_flash, summarizer_prompt, reviewer_prompt, debug=True
)

Image(summary_chatbot.agent_graph.get_graph().draw_mermaid_png())

In [ ]:
loader = PyPDFLoader("./data/renewable_energy.pdf")
docs = loader.load()
source_content = docs[0].page_content.replace("\n", " ")
print(source_content)

In [ ]:
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

messages=[HumanMessage(content=source_content)]
result=summary_chatbot.agent_graph.invoke({"messages":messages},config)

In [ ]:
summary_chatbot = SummaryAgent(
    model_gemini_2_0_flash, summarizer_prompt, reviewer_prompt, debug=False
)


user_inputs = [
    source_content,
    "Can you rewrite the review making it more scientific?",
    "Can you make it even shorter?",
]

# Create a new thread
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

# Given the number of iterations, this will take a long time.
for input in user_inputs:
    print(f"----------------------------------------\nUSER : {input}")
    # Format the user message
    user_message = {"messages": [HumanMessage(input)]}
    # Get response from the agent
    ai_response = summary_chatbot.agent_graph.invoke(
        user_message, config=config
    )
    # Print the response
    print(f"\nAGENT : {ai_response['messages'][-1].content}")